# **Imports and Functions**

In [ ]:
"""
================================= DATA CREATION UTILS =================================

# Requirements (install once)
Python >= 3.9

pip install numpy opencv-python tifffile matplotlib pillow scipy

# Optional (for development)
pip install black ruff mypy
=======================================================================================
"""

import os
import numpy as np
import cv2
import tifffile as tiff
import matplotlib.pyplot as plt
from PIL import Image  # kept for parity with original imports (even if unused)
from scipy.ndimage import binary_dilation


# ----------------------------------------------------------------------------------------------
# Segmentation reader
# ----------------------------------------------------------------------------------------------
def tif_read_seg(
    tiff_paths,
    crop_size=256,
    transpose_first_n=None,
    max_frames=None,
    num_crops=49,
    start_indices=None
):
    """
    Read segmentation TIFF(s), split into square crops, optionally transpose the first N frames,
    and mark isolated background regions as 'undefined' (label=3) when they are not adjacent to cells.

    Args:
        tiff_paths (str | list[str]): Path or list of paths to segmentation TIFF files.
        crop_size (int): Size of each square crop (pixels). Default: 256.
        transpose_first_n (int | float | list[int|float] | None):
            If provided, transpose (cv2.transpose) frames whose sequential counter < this limit.
            Can be a single value or list per file. If None, treated as +inf (no transpose limit).
        max_frames (int | list[int] | None): Max frames to read per file (after start index). If None -> all.
        num_crops (int): Number of crops per full frame; must be a perfect square (tiles^2).
        start_indices (list[int] | None): Start frame index per file; default 0 for each.

    Returns:
        np.ndarray: Cropped segmentation frames (each crop is a 2D array).
        list[int]: Indices of black (all-zero) frames in the sequential counter (to skip elsewhere).
    """

    def separate_undefined_regions(frame):
        # (Preserved logic) mark zero areas far from cells as class 3 (undefined)
        frame = frame.copy()
        zero_mask = (frame == 0)
        cell_mask = (frame == 1) | (frame == 2)
        dilated_cells = binary_dilation(cell_mask)
        undefined_mask = zero_mask & (~dilated_cells)
        frame[undefined_mask] = 3
        return frame

    # --- Preserve original input normalization logic ---
    if isinstance(tiff_paths, str):
        tiff_paths = [tiff_paths]
    if isinstance(max_frames, int):
        max_frames = [max_frames] * len(tiff_paths)
    if max_frames is None:
        max_frames = [None] * len(tiff_paths)
    if start_indices is None:
        start_indices = [0] * len(tiff_paths)
    if transpose_first_n is None:
        transpose_first_n = [float('inf')] * len(tiff_paths)
    elif isinstance(transpose_first_n, (int, float)):
        transpose_first_n = [transpose_first_n] * len(tiff_paths)

    tiles = int(np.sqrt(num_crops))
    assert tiles**2 == num_crops

    black_indices = []
    frames = []
    frame_counter = 0

    for path_idx, tiff_path in enumerate(tiff_paths):
        max_local = max_frames[path_idx]
        start_local = start_indices[path_idx]
        transpose_limit = transpose_first_n[path_idx]

        with tiff.TiffFile(tiff_path) as tif:
            # odd pages only
            all_pages = list(range(1, len(tif.pages), 2))
            all_pages = all_pages[start_local:]
            if max_local is not None:
                all_pages = all_pages[:max_local]

            for i in all_pages:
                frame = tif.pages[i].asarray()
                if np.all(frame == 0):
                    black_indices.append(frame_counter)
                    frame_counter += 1
                    continue

                if frame_counter < transpose_limit:
                    frame = cv2.transpose(frame)

                for gy in range(tiles):
                    for gx in range(tiles):
                        y = gy * crop_size
                        x = gx * crop_size
                        crop = frame[y:y+crop_size, x:x+crop_size]
                        frames.append(separate_undefined_regions(crop))

                frame_counter += 1

    return np.array(frames), black_indices


# ----------------------------------------------------------------------------------------------
# Data reader (two channels)
# ----------------------------------------------------------------------------------------------
def tif_read_data(
    tiff_paths,
    crop_size=256,
    max_frames=None,
    skip_indices=None,
    num_crops=49,
    start_indices=None
):
    """
    Read data TIFF(s) that contain alternating channels per frame (ch1 at even page, ch2 at odd page),
    split each frame into crops, and skip frames by global frame counter indices if requested.

    Args:
        tiff_paths (str | list[str]): Path or list of paths to data TIFF files.
        crop_size (int): Size of each square crop (pixels). Default: 256.
        max_frames (int | list[int] | None): Max frames to read per file; if None -> all.
        skip_indices (list[int] | None): Global frame indices to skip (e.g., black frames from seg).
        num_crops (int): Number of crops per full frame; must be a perfect square (tiles^2).
        start_indices (list[int] | None): Start frame index per file; default 0 for each.

    Returns:
        np.ndarray: Crops of ch1 and ch2 appended sequentially: [ch1_crop0, ch2_crop0, ch1_crop1, ch2_crop1, ...].

    Notes:
        - Logic preserved: ch1 = pages[2*i], ch2 = pages[2*i+1].
        - Global frame counter increments even when skipping indices.
    """
    if isinstance(tiff_paths, str):
        tiff_paths = [tiff_paths]
    if skip_indices is None:
        skip_indices = []
    if isinstance(max_frames, int):
        max_frames = [max_frames] * len(tiff_paths)
    if max_frames is None:
        max_frames = [None] * len(tiff_paths)
    if start_indices is None:
        start_indices = [0] * len(tiff_paths)

    tiles = int(np.sqrt(num_crops))
    assert tiles**2 == num_crops

    frames = []
    frame_counter = 0

    for path_idx, tiff_path in enumerate(tiff_paths):
        max_local = max_frames[path_idx]
        start_local = start_indices[path_idx]
        with tiff.TiffFile(tiff_path) as tif:
            total_frames = len(tif.pages) // 2
            all_indices = list(range(start_local, total_frames))
            if max_local is not None:
                all_indices = all_indices[:max_local]

            for i in all_indices:
                if frame_counter in skip_indices:
                    frame_counter += 1
                    continue

                ch1 = tif.pages[2*i].asarray()
                ch2 = tif.pages[2*i+1].asarray()

                for gy in range(tiles):
                    for gx in range(tiles):
                        y = gy * crop_size
                        x = gx * crop_size
                        frames.append(ch1[y:y+crop_size, x:x+crop_size])
                        frames.append(ch2[y:y+crop_size, x:x+crop_size])

                frame_counter += 1

    return np.array(frames)


# ----------------------------------------------------------------------------------------------
# Position (RGB/single-channel) reader
# ----------------------------------------------------------------------------------------------
def tif_read_position(
    tiff_paths,
    crop_size=256,
    max_frames=None,
    skip_indices=None,
    num_crops=49,
    start_indices=None
):
    """
    Read position TIFF(s) (single-channel per page), split into crops, and skip frames by global counter.

    Args:
        tiff_paths (str | list[str]): Path or list of paths to position TIFF files.
        crop_size (int): Size of each square crop (pixels). Default: 256.
        max_frames (int | list[int] | None): Max frames to read per file; if None -> all.
        skip_indices (list[int] | None): Global frame indices to skip.
        num_crops (int): Number of crops per full frame; must be a perfect square (tiles^2).
        start_indices (list[int] | None): Start frame index per file; default 0 for each.

    Returns:
        np.ndarray: Cropped position frames.
    """
    if isinstance(tiff_paths, str):
        tiff_paths = [tiff_paths]
    if skip_indices is None:
        skip_indices = []
    if isinstance(max_frames, int):
        max_frames = [max_frames] * len(tiff_paths)
    if max_frames is None:
        max_frames = [None] * len(tiff_paths)
    if start_indices is None:
        start_indices = [0] * len(tiff_paths)

    tiles = int(np.sqrt(num_crops))
    assert tiles**2 == num_crops

    frames = []
    frame_counter = 0

    for path_idx, tiff_path in enumerate(tiff_paths):
        max_local = max_frames[path_idx]
        start_local = start_indices[path_idx]
        with tiff.TiffFile(tiff_path) as tif:
            total_frames = len(tif.pages)
            all_indices = list(range(start_local, total_frames))
            if max_local is not None:
                all_indices = all_indices[:max_local]

            for i in all_indices:
                if frame_counter in skip_indices:
                    frame_counter += 1
                    continue

                frame = tif.pages[i].asarray()

                for gy in range(tiles):
                    for gx in range(tiles):
                        y = gy * crop_size
                        x = gx * crop_size
                        frames.append(frame[y:y+crop_size, x:x+crop_size])

                frame_counter += 1

    return np.array(frames)


# ----------------------------------------------------------------------------------------------
# Dataset loader (combines the three readers)
# ----------------------------------------------------------------------------------------------
def load_dataset(
    seg_paths,
    data_paths,
    rgb_paths,
    crop_size=256,
    frames_per_file=None,
    num_crops=49,
    transpose_first_n=None,
    start_indices=None
):
    """
    Load the dataset by reading segmentation, data (2 channels), and RGB/position TIFFs.
    Black (all-zero) segmentation frames are recorded and skipped in data/RGB accordingly.

    Args:
        seg_paths (list[str] | str): Segmentation TIFF path(s).
        data_paths (list[str] | str): Data TIFF path(s) (interleaved ch1/ch2).
        rgb_paths (list[str] | str): Position/RGB TIFF path(s).
        crop_size (int): Crop size (pixels). Default: 256.
        frames_per_file (list[int] | int | None): Max frames per file. If int, applies to all.
        num_crops (int): Number of crops per frame (tiles^2). Must be a perfect square.
        transpose_first_n (list[int|float] | int | float | None): See tif_read_seg docstring.
        start_indices (list[int] | None): Start frame index per file.

    Returns:
        tuple:
            seg (np.ndarray): Segmentation crops.
            data (np.ndarray): Interleaved data crops [ch1_0, ch2_0, ch1_1, ch2_1, ...].
            rgb (np.ndarray): RGB/position crops.
            black (list[int]): Global black frame indices detected in segmentation.
    """
    seg, black = tif_read_seg(
        seg_paths,
        crop_size=crop_size,
        max_frames=frames_per_file,
        num_crops=num_crops,
        transpose_first_n=transpose_first_n,
        start_indices=start_indices
    )

    data = tif_read_data(
        data_paths,
        crop_size=crop_size,
        max_frames=frames_per_file,
        skip_indices=black,
        num_crops=num_crops,
        start_indices=start_indices
    )

    rgb = tif_read_position(
        rgb_paths,
        crop_size=crop_size,
        max_frames=frames_per_file,
        skip_indices=black,
        num_crops=num_crops,
        start_indices=start_indices
    )

    assert len(seg) == len(rgb) == len(data) // 2, \
        f"Mismatch: seg={len(seg)}, rgb={len(rgb)}, data={len(data)//2}"

    return seg, data, rgb, black


# ----------------------------------------------------------------------------------------------
# Saving utilities
# ----------------------------------------------------------------------------------------------
def save_data(frames, output_folder="data", start_index=0):
    """
    Save paired channel crops to individual TIFFs:
        frame_00000_ch1.tif, frame_00000_ch2.tif, frame_00001_ch1.tif, ...

    Args:
        frames (np.ndarray): Interleaved crops [ch1_0, ch2_0, ch1_1, ch2_1, ...].
        output_folder (str): Target directory for saved TIFFs.
        start_index (int): Starting frame index for naming.

    Raises:
        ValueError: If number of frames is not even.
    """
    if len(frames) % 2 != 0:
        raise ValueError("Number of frames must be even (pairs of ch1 and ch2).")

    os.makedirs(output_folder, exist_ok=True)

    for i in range(0, len(frames), 2):
        ch1 = frames[i]
        ch2 = frames[i + 1]
        frame_num = start_index + (i // 2)

        tiff.imwrite(f"{output_folder}/frame_{frame_num:05d}_ch1.tif", ch1)
        tiff.imwrite(f"{output_folder}/frame_{frame_num:05d}_ch2.tif", ch2)


def save_seg(frames, output_folder="label", start_index=0):
    """
    Save segmentation crops to TIFFs after visual scaling to [0..255] by (val/3)*255.

    Args:
        frames (np.ndarray): Segmentation crops with labels {0,1,2,3}.
        output_folder (str): Target directory.
        start_index (int): Starting index for naming.
    """
    os.makedirs(output_folder, exist_ok=True)
    for i, frame in enumerate(frames):
        frame_num = start_index + i

        # Normalize to 0–255 assuming max label=3 (preserved logic)
        frame_vis = (frame.astype(np.float32) / 3 * 255).astype(np.uint8)

        tiff.imwrite(f"{output_folder}/frame_{frame_num:05d}.tif", frame_vis)


def save_RGB(frames, output_folder="RGB", start_index=0):
    """
    Save RGB/position crops to TIFFs.

    Args:
        frames (np.ndarray): RGB/position crops (single channel or RGB).
        output_folder (str): Target directory.
        start_index (int): Starting index for naming.
    """
    os.makedirs(output_folder, exist_ok=True)
    for i, frame in enumerate(frames):
        frame_num = start_index + i
        tiff.imwrite(f"{output_folder}/frame_{frame_num:05d}.tif", frame)


# ----------------------------------------------------------------------------------------------
# Filtering utilities
# ----------------------------------------------------------------------------------------------
def is_mostly_undefined(crop, threshold_ratio=0.3):
    """
    Return True if more than 'threshold_ratio' of the pixels are label==3 (undefined).

    Args:
        crop (np.ndarray): A single segmentation crop.
        threshold_ratio (float): Fraction threshold for undefined pixels.

    Returns:
        bool: True if undefined ratio > threshold_ratio.
    """
    undefined_pixels = np.sum(crop == 3)
    total_pixels = crop.size
    return (undefined_pixels / total_pixels) > threshold_ratio


def filter_valid_frames(seg, data, rgb, threshold_ratio=0.3):
    """
    Filter out crops whose segmentation is mostly undefined (label 3), keeping data and rgb aligned.

    Args:
        seg (np.ndarray): Segmentation crops.
        data (np.ndarray): Interleaved data crops [ch1_0, ch2_0, ...].
        rgb (np.ndarray): RGB/position crops.
        threshold_ratio (float): Fraction threshold used by is_mostly_undefined.

    Returns:
        tuple:
            valid_seg (np.ndarray)
            valid_data (np.ndarray)
            valid_rgb (np.ndarray)

    Raises:
        AssertionError: If 'data' length is not twice the 'seg' length.
    """
    valid_seg = []
    valid_data = []
    valid_rgb = []

    assert len(data) == 2 * len(seg), "data צריך להכיל ch1 ו-ch2 לכל crop"

    for i in range(len(seg)):
        seg_crop = seg[i]
        if is_mostly_undefined(seg_crop, threshold_ratio):
            continue

        valid_seg.append(seg_crop)
        valid_data.append(data[2 * i])
        valid_data.append(data[2 * i + 1])
        valid_rgb.append(rgb[i])

    return np.array(valid_seg), np.array(valid_data), np.array(valid_rgb)


# ----------------------------------------------------------------------------------------------
# Visualization helper
# ----------------------------------------------------------------------------------------------
def show_frame_rows(seg, rgb, data, num_rows=5, start_index=0):
    """
    Display rows of (Segmentation, RGB, Channel1, Channel2) for quick visual inspection.

    Args:
        seg (np.ndarray): Segmentation crops.
        rgb (np.ndarray): RGB/position crops.
        data (np.ndarray): Interleaved data crops [ch1_0, ch2_0, ...].
        num_rows (int): Number of rows (frames) to display.
        start_index (int): Starting frame index.

    Returns:
        None
    """
    num_total = min(len(seg), len(rgb), len(data) // 2)
    end_index = min(start_index + num_rows, num_total)

    for i in range(start_index, end_index):
        fig, axs = plt.subplots(1, 4, figsize=(12, 3))
        fig.suptitle(f"Frame {i}", fontsize=14)

        axs[0].imshow(seg[i], cmap='tab10')
        axs[0].set_title("Segmentation")

        axs[1].imshow(rgb[i], cmap='gray')
        axs[1].set_title("RGB")

        axs[2].imshow(data[2 * i], cmap='gray')
        axs[2].set_title("Channel 1")

        axs[3].imshow(data[2 * i + 1], cmap='gray')
        axs[3].set_title("Channel 2")

        for ax in axs:
            ax.axis('off')

        plt.tight_layout()
        plt.show()

# **Example**

In [ ]:
# ==============================================================================================
# EXAMPLE USAGE (COMMENTED): Adjust paths as needed. No notebook magics, works anywhere.
# ==============================================================================================
# seg_paths  = [
#     "position2_seg_P0_310722.tif",
#     "position3_seg_P0_310722.tif",
#     "position3_seg_E17.5_111121.tif",
#     "position4_seg_E17.5_111121.tif",
#     "position2_seg_P0_050622.tif",
#     "position3_seg_E17.5_200122.tif",
#     "position3_seg_P0_250623.tif",
#     "position2_seg_E17.5_011222.tif",
#     "position1_seg_200222.tif"
# ]
#
# data_paths = [
#     "position2_P0_310722.tif",
#     "position3_P0_310722.tif",
#     "position3_E17.5_111121.tif",
#     "position4_E17.5_111121.tif",
#     "position2_P0_050622.tif",
#     "position3_E17.5_200122.tif",
#     "position3_P0_250623.tif",
#     "position2_E17.5_011222.tif",
#     "position1_200222.tif"
# ]
#
# rgb_paths = [
#     "position2_RGB_P0_310722.tif",
#     "position3_RGB_P0_310722.tif",
#     "position3_RGB_E17.5_111121.tif",
#     "position4_RGB_E17.5_111121.tif",
#     "position2_RGB_P0_050622.tif",
#     "position3_RGB_E17.5_200122.tif",
#     "position3_RGB_P0_250623.tif",
#     "position2_RGB_E17.5_011222.tif",
#     "position1_RGB_200222.tif"
# ]
#
# seg, data, rgb, black = load_dataset(
#     seg_paths=seg_paths,
#     data_paths=data_paths,
#     rgb_paths=rgb_paths,
#     crop_size=256,
#     frames_per_file=[100,100,100,100,100,100,100,100,100],
#     num_crops=49,
#     transpose_first_n=[10,float('inf'),float('inf'),float('inf'),float('inf'),float('inf'),float('inf'),float('inf'),float('inf')],
#     start_indices=[0,0,0,0,0,0,0,0,0]
# )
#
# seg_filtered, data_filtered, rgb_filtered = filter_valid_frames(seg, data, rgb, threshold_ratio=0.3)
#
# # Choose an output root (relative or absolute). No %cd needed.
# out_root = "./training_data/Full_DataSet"
# os.makedirs(out_root, exist_ok=True)
# save_data(data_filtered, os.path.join(out_root, "data"), start_index=0)
# save_seg(seg_filtered, os.path.join(out_root, "label"), start_index=0)
# save_RGB(rgb_filtered, os.path.join(out_root, "RGB"), start_index=0)
#
# print(f"Saved {len(seg_filtered)} frames out of {len(seg)} after filtering.")
# show_frame_rows(seg_filtered, rgb_filtered, data_filtered, num_rows=2, start_index=0)